In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH06-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH06-Memory


# VectorStoreRetrieverMemory

1. 개요
    - 대화내용을 Vector Store에 저장하고 임베딩 기반 검색 통해 대화 문맥 관리 
    - 유사성 기반으로 적합한 문맥을 검색하여 대화 문맥을 유지하고 자연스러운 응답 생성에 활용

2. 주요 특징 
    1. Vector Store 저장:
        - 대화 데이터를 텍스트 임베딩 하여 저장  
        - 다양한 Vector Store(FAISS, Pinecone, Weaviate 등)과 통합 가능 
        - 긴 대화 기록을 저장하고 필요할 때 검색하여 대화 흐름 일관성 유지 (장기 문맥 관리)  
    2. 임베딩 기반 검색:
        - 대화 문맥이나 질문에 가장 유사한 대화를 검색하여 활용  
        - 단순 키워드 검색 아닌 문맥 기반 검색 

[Reference] https://api.python.langchain.com/en/latest/memory/langchain.memory.vectorstore.VectorStoreRetrieverMemory.html

In [4]:
import faiss
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [10]:
# 임베딩 모델 정의 
embedding_model = OpenAIEmbeddings()

# Vector Store 정의/초기화
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embedding_model, index, InMemoryDocstore({}), {})

# 메모리 정의/초기화 
retriever = vectorstore.as_retriever(search_kwargs={"k":1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [11]:
# 대화 내용 저장
memory.save_context(
    inputs={
        "human": "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
    },
    outputs={
        "ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."
    },
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={
        "ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."
    },
)
memory.save_context(
    inputs={
        "human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
    },
    outputs={
        "ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었습니다. 이를 해결하기 위해 저희 팀은 정기적인 미팅을 갖고 각자의 진행 상황을 공유했습니다. 또한, 문제가 발생했을 때는 적극적으로 의견을 나누고, 합리적인 해결책을 찾기 위해 노력했습니다."
    },
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={
        "ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다."
    },
)

In [12]:
# 저장된 메모리 확인
# k=1
print(memory.load_memory_variables({"human": "면접자 전공은 무엇인가요?"})["history"])

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.


In [14]:
print(
    memory.load_memory_variables(
        {"human": "면접자가 프로젝트에서 맡은 역할은 PM 인가요?"}
    )["history"]
)

human: 프로젝트에서 어떤 역할을 맡았나요?
ai: 제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다.


In [15]:
print(
    memory.load_memory_variables(
        {"human": "면접자의 취미는 무엇인가요?"}
    )["history"]
)

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.


-----
** End of Documents **